In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_csv('./Datasets/alzheimers_prediction_dataset_usa.csv')

In [3]:
df.columns

Index(['Unnamed: 0', 'Country', 'Age', 'Gender', 'Education Level', 'BMI',
       'Physical Activity Level', 'Smoking Status', 'Alcohol Consumption',
       'Diabetes', 'Hypertension', 'Cholesterol Level',
       'Family History of Alzheimer’s', 'Cognitive Test Score',
       'Depression Level', 'Sleep Quality', 'Dietary Habits',
       'Air Pollution Exposure', 'Employment Status', 'Marital Status',
       'Genetic Risk Factor (APOE-ε4 allele)', 'Social Engagement Level',
       'Income Level', 'Stress Levels', 'Urban vs Rural Living',
       'Alzheimer's Diagnosis'],
      dtype='object')

In [6]:
# Define the target column
target_column = "Alzheimer's Diagnosis"

# Define the feature columns (all the columns except the target column)
feature_columns = [
    'Age', 'BMI', 'Education Level', 'Physical Activity Level', 'Smoking Status', 
    'Alcohol Consumption', 'Diabetes', 'Hypertension', 'Cholesterol Level', 
    'Family History of Alzheimer’s', 'Cognitive Test Score', 'Depression Level', 
    'Sleep Quality', 'Dietary Habits', 'Air Pollution Exposure', 'Employment Status', 
    'Marital Status', 'Genetic Risk Factor (APOE-ε4 allele)', 'Social Engagement Level', 
    'Income Level', 'Stress Levels', 'Urban vs Rural Living'
]

# Get the features (X) and target (y)
X = df[feature_columns]  # Features dataframe
y = df[target_column]    # Target variable

# Display the features and target
print("Features (X):")
X.head()


Features (X):


,Age,BMI,Education Level,Physical Activity Level,Smoking Status,Alcohol Consumption,Diabetes,Hypertension,Cholesterol Level,Family History of Alzheimer’s,...,Sleep Quality,Dietary Habits,Air Pollution Exposure,Employment Status,Marital Status,Genetic Risk Factor (APOE-ε4 allele),Social Engagement Level,Income Level,Stress Levels,Urban vs Rural Living
0,61,33.1,13,Medium,Former,Never,No,Yes,High,No,...,Good,Healthy,High,Employed,Married,No,High,Low,Low,Urban
1,55,29.9,16,Medium,Former,Occasionally,No,No,High,No,...,Poor,Healthy,Low,Retired,Widowed,No,Low,Medium,Low,Rural
2,72,31.5,2,Low,Former,Regularly,No,No,Normal,No,...,Poor,Average,High,Employed,Widowed,No,High,High,Medium,Urban
3,57,30.7,0,Low,Current,Regularly,No,Yes,High,Yes,...,Good,Unhealthy,Low,Employed,Single,No,High,Medium,Medium,Rural
4,69,19.7,12,High,Former,Regularly,No,No,High,No,...,Good,Average,Medium,Retired,Married,No,Low,Medium,Low,Urban


In [7]:
print("\nTarget (y):")
y.head()


Target (y):


0    No
1    No
2    No
3    No
4    No
Name: Alzheimer's Diagnosis, dtype: object